In [1]:
def postgame_report(remove_raw_data,
                    slash = "\\",
                    data ='Raw.csv', 
                    path ='C:\\Users\\bmmcc\\OneDrive\\Desktop\\2021 Regular Season\\',
                    home = 'ALA_CRI',
                    scrimmage_orig = 'ALA_CRI1',
                    scrimmage_new = 'ALA_SCRIM',
                    date = 'Date',
                    tagged = 'TaggedPitchType',
                    PitchNo = 'PitchNo',
                    pitchcall = 'PitchCall',
                    batter_team = 'BatterTeam',
                    pitcher_team = 'PitcherTeam',
                    pitcher = 'Pitcher',
                    batter = 'Batter',
                    velo = 'RelSpeed',
                    spin = 'SpinRate',
                    axis = 'SpinAxis',
                    tilt = 'Tilt',
                    IVB = 'InducedVertBreak',
                    HB = 'HorzBreak',
                    VAA = 'VertApprAngle',
                    HAA = 'HorzApprAngle',
                    rel_height = 'RelHeight',
                    rel_side = 'RelSide',
                    extension = 'Extension',
                    pfxx = 'pfxx',
                    pfxz = 'pfxz',
                    exitspeed = 'ExitSpeed',
                    exitangle = 'Angle',
                    exitdirection = 'Direction',
                    hitspin = 'HitSpinRate',
                    distance = 'Distance',
                    hangtime = 'HangTime',
                    hardhit = 'HardHit',
                    PlateLocHeight = 'PlateLocHeight',
                    PlateLocSide = 'PlateLocSide',
                    Zone = 'Zone'
                    ):
    
    # ---------- Import Libraries ----------
    import os
    import csv
    import shutil
    import scipy
    import pandas as pd
    import numpy as np
    import seaborn as sns
    import matplotlib.pyplot as plt
    from PyPDF2 import PdfFileMerger
    from scipy.interpolate import griddata
    sns.set_style('whitegrid')

    # ---------- Import CSV and Clean ----------
    df = pd.read_csv(data).replace(0, np.nan).round(2)
    df = df[df[tagged] != 'Undefined']
    df[tagged] = df[tagged].replace(['Fastball'],'4-Fastball')
    df[tagged] = df[tagged].replace(['Sinker'],'2-Fastball')
    df[hardhit] = pd.cut(df[exitspeed],bins=[0,95,2000], labels=[0,1]).astype(str)
    
    # ---------- Make In/Out Zone call ----------
    df2 = []
    for index, row in df.iterrows():
        if row[PlateLocHeight] < 1.5:
            df2.append('Out')
        elif row[PlateLocHeight] > 3.6:
            df2.append('Out')
        elif row[PlateLocSide] < -2/3:
            df2.append('Out')
        elif row[PlateLocSide] > 2/3:    
            df2.append('Out')
        else:
            df2.append('In')
    df[Zone] = df2

    # ---------- Create Confusion Matrix Variables ----------
    df3 = []
    for index, row in df.iterrows():
        # Out of Zone
        if row['Zone'] == 'Out':
            # Swung Out of Zone
            if row['PitchCall'] == 'StrikeSwinging':
                df3.append(1) 
            elif row['PitchCall'] == 'FoulBall':
                df3.append(1) 
            elif row['PitchCall'] == 'InPlay':
                df3.append(1) 
            # No Swing Out Of Zone
            elif row['PitchCall'] == 'BallCalled':
                df3.append(3)         
            elif row['PitchCall'] == 'StrikeCalled':
                df3.append(3) 
            #Else / Hit by Pitch
            else:
                df3.append(5)
        # In the Zone
        elif row['Zone'] == 'In':
            # Swung In Zone
            if row['PitchCall'] == 'StrikeSwinging':
                df3.append(2) 
            elif row['PitchCall'] == 'FoulBall':
                df3.append(2) 
            elif row['PitchCall'] == 'InPlay':
                df3.append(2) 
            # No Swing Out Of Zone
            elif row['PitchCall'] == 'BallCalled':
                df3.append(4)         
            elif row['PitchCall'] == 'StrikeCalled':
                df3.append(4)
            # Else / Hit by Pitch
            else:
                df3.append(5)
        else:
            df3.append('5')
    df['confusion'] = df3
    
    df = pd.concat([df, pd.get_dummies(df['confusion'])], axis=1)
    df = df.drop(columns = ['confusion'])
    
    # ---------- Generate Opp ID --------
    if df[batter_team][0] == home: 
        if df[pitcher_team][0] != scrimmage_orig: 
            opp_id = df[pitcher_team][0]
        elif df[pitcher_team][0] == scrimmage_orig: 
            opp_id = df[batter_team][0]
    
    if df[pitcher_team][0] == home: 
        if df[batter_team][0] == scrimmage_orig:
            opp_id = df[pitcher_team][0]
            
    # ---------- Convert to scrimmage code (if needed) -----------
    if df[batter_team][0] == scrimmage_orig: 
        df[batter_team] = df[batter_team].replace([scrimmage_orig],scrimmage_new)
        df[pitcher_team] = df[pitcher_team].replace([scrimmage_orig],scrimmage_new)
    else: pass

    # ---------- Create Game ID and Game Folder ---------
    game_id = str(df[batter_team].reset_index(drop = True)[0]) + ' (' + str(pd.to_datetime(df[date][0]).date()) + ')'
    shutil.copytree('Game_Summary - Temp', game_id)
    
    # ---------- PDF endings ----------
    pdf_endings_pitching = ['a.pdf', 'b.pdf', 'c.pdf', 'd.pdf', 'e.pdf', 'f.pdf', 'g.pdf']
    pdf_endings_hitting = ['a.pdf', 'b.pdf', 'c.pdf']
    pdf_endings_file = [game_id + ' - Strike Zone.pdf', game_id + ' - ALA Strike Zone.pdf', game_id + ' - Opp. Strike Zone.pdf']
    
    
    # ---------- Create Alabama and Opponent Dataframes -----------
    df_opponent = df[df[pitcher_team].str.contains(home) == False]
    df_pitches = df[tagged].unique().tolist()

    # ---------- Pitcher / Hitter indexs (scrimmages or games)] --------
    if df[batter_team][1] == scrimmage_new: 
        df_alabama = df
    else:
        df_alabama = df[df[pitcher_team].str.contains(home)]

    if df[batter_team][0] == scrimmage_new: 
        df_opponent = df
    else:
        df_opponent = df[df[pitcher_team].str.contains(home) == False] 

    if df[batter_team][0] == scrimmage_new: 
        df_alabama_hitter_index = df[batter].unique().tolist()
    else: 
        df_alabama_hitter_index = df_opponent[batter].unique().tolist()
        df_opponent_hitter_index = df_opponent[batter].unique().tolist()

    if df[batter_team][0] == scrimmage_new: 
        df_alabama_pitchers_index = df[pitcher].unique().tolist()
    else:
        df_alabama_pitchers_index = df_alabama[pitcher].unique().tolist()
        df_opp_pitchers_index = df_opponent[pitcher].unique().tolist() 


    # ---------- Create summary reports and send to CSV File---------
    df_alabama_pitches = df_alabama.groupby([pitcher, tagged])[[velo, spin, axis, IVB, HB, VAA, HAA, rel_height, rel_side, extension, pfxx, pfxz]].mean()
    df_opponent_pitches = df_opponent.groupby([pitcher, tagged])[[velo, spin, axis, IVB, HB, VAA, HAA, rel_height, rel_side, extension, pfxx, pfxz]].mean()
    df_alabama_hitting = df_opponent.groupby([batter, tagged])[[exitspeed, exitangle, exitdirection, hitspin, distance, hangtime]].mean()
    df_opponent_hitting = df_alabama.groupby([batter, tagged])[[exitspeed, exitangle, exitdirection, hitspin, distance, hangtime]].mean()

    df_alabama_pitches.to_csv(game_id + slash + 'Pitching' + slash + 'Summary' + slash + game_id + ' -- Alabama Pitching Summary.csv')
    df_opponent_pitches.to_csv(game_id + slash + 'Hitting' + slash + 'Summary' + slash + game_id + ' -- Opponent Pitching Summary.csv')
    df_opponent_hitting.to_csv(game_id + slash + 'Pitching' + slash + 'Summary' + slash + game_id + ' -- Opponent Batting Summary.csv')
    df_alabama_hitting.to_csv(game_id + slash + 'Hitting' + slash + 'Summary' + slash + game_id + ' -- Alabama Batting Summary.csv')

    # ---------- Individual Reports --------- 
    name_count = 1
    for name in df_alabama_pitchers_index:
        inv_pitch = df_alabama[df_alabama[pitcher].str.contains(name)]
        df_alabama_pitches = inv_pitch.groupby([tagged])[[velo, spin, axis, IVB, HB, VAA, HAA, rel_height, rel_side, extension, pfxx, pfxz]].mean()
        df_alabama_pitches.to_csv(game_id + slash + 'Pitching' + slash + 'pitcher_' + str(name_count) + slash + name + ' Summary.csv')
        os.rename(game_id + slash + 'Pitching' + slash + 'pitcher_' + str(name_count), game_id + slash + 'Pitching' + slash + name)    
        name_count += 1

    name_count = 1
    for name in df_alabama_hitter_index:
        os.rename(game_id + slash + 'Hitting' + slash + 'batter_' + str(name_count), game_id + slash + 'Hitting' + slash + name)    
        name_count += 1

    # ---------- Strike Zone Report ---------
    frame = df[[PlateLocHeight, PlateLocSide, pitchcall]]
    df_ball = frame[frame[pitchcall].str.contains('BallCalled')]
    df_strike = frame[frame[pitchcall].str.contains('StrikeCalled')]
    plt.scatter(df_ball[PlateLocSide], df_ball[PlateLocHeight], label='Ball Called')
    plt.scatter(df_strike[PlateLocSide], df_strike[PlateLocHeight], label='Strike Called')
    plt.axhline(y=1.5, linewidth=2, color='k')
    plt.axhline(y=3.6, linewidth=2, color='k')
    plt.axvline(x=(2/3), linewidth=2, color='k')
    plt.axvline(x=(-2/3), linewidth=2, color='k')
    plt.legend()
    plt.title(game_id + '\nBall / Strike Zone Chart')
    plt.xlabel('<L>     Side in Feet     <R>')
    plt.ylabel('Height in Feet')
    plt.xlim([-2.5, 2.5])
    plt.ylim([0, 5]) 
    plt.savefig(game_id + slash + 'Game' + slash + game_id + ' - Strike Zone.pdf')
    plt.show()
    
    # ---------- Alabama Strike Zone Report ---------
    frame = df[[PlateLocHeight, PlateLocSide, pitchcall]]
    df_ball = df_alabama[df_alabama[pitchcall].str.contains('BallCalled')]
    df_strike = df_alabama[df_alabama[pitchcall].str.contains('StrikeCalled')]
    plt.scatter(df_ball[PlateLocSide], df_ball[PlateLocHeight], label='Ball Called')
    plt.scatter(df_strike[PlateLocSide], df_strike[PlateLocHeight], label='Strike Called')
    plt.axhline(y=1.5, linewidth=2, color='k')
    plt.axhline(y=3.6, linewidth=2, color='k')
    plt.axvline(x=(2/3), linewidth=2, color='k')
    plt.axvline(x=(-2/3), linewidth=2, color='k')
    plt.legend()
    plt.title('Alabama Pitching - ' + game_id + '\nBall / Strike Zone Chart')
    plt.xlabel('<L>     Side in Feet     <R>')
    plt.ylabel('Height in Feet')
    plt.xlim([-2.5, 2.5])
    plt.ylim([0, 5]) 
    plt.savefig(game_id + slash + 'Game' + slash + game_id + ' - ALA Strike Zone.pdf')
    plt.show()

    # ---------- Opponent Strike Zone Report ---------
    frame = df[[PlateLocHeight, PlateLocSide, pitchcall]]
    df_ball = df_opponent[df_opponent[pitchcall].str.contains('BallCalled')]
    df_strike = df_opponent[df_opponent[pitchcall].str.contains('StrikeCalled')]
    plt.scatter(df_ball[PlateLocSide], df_ball[PlateLocHeight], label='Ball Called')
    plt.scatter(df_strike[PlateLocSide], df_strike[PlateLocHeight], label='Strike Called')
    plt.axhline(y=1.5, linewidth=2, color='k')
    plt.axhline(y=3.6, linewidth=2, color='k')
    plt.axvline(x=(2/3), linewidth=2, color='k')
    plt.axvline(x=(-2/3), linewidth=2, color='k')
    plt.legend()
    plt.title('Opponent Pitching - ' + game_id + '\nBall / Strike Zone Chart')
    plt.xlabel('<L>     Side in Feet     <R>')
    plt.ylabel('Height in Feet')
    plt.xlim([-2.5, 2.5])
    plt.ylim([0, 5]) 
    plt.savefig(game_id + slash + 'Game' + slash + game_id + ' - Opp. Strike Zone.pdf')
    plt.show()
    
    # ---------- Velo Trend Report ---------
    for name in df_alabama_pitchers_index:
        for pitch in df_pitches:
            df_pitch_spec_name = df[df[pitcher].str.contains(name)]
            df_pitch_spec_pitch = df_pitch_spec_name[df_pitch_spec_name[tagged].str.contains(pitch)]
            df_pitch_spec = df_pitch_spec_pitch[velo]
            df_pitch_spec = df_pitch_spec.reset_index()
            df_pitch_spec = df_pitch_spec.drop(['index'], axis=1)
            plt.plot(df_pitch_spec, label = pitch)
            plt.xlabel('# of pitches thrown')
            plt.ylabel('MPH')
            plt.title(name + '\nVelo (in MPH)')
            plt.legend()
        plt.savefig(game_id + slash + 'Pitching' + slash + name + slash + name + pdf_endings_pitching[0], bbox_inches='tight')
        plt.show()
        
    # ---------- SpinRate Trend Report ---------
    for name in df_alabama_pitchers_index:
        for pitch in df_pitches:
            df_pitch_spec_name = df[df[pitcher].str.contains(name)]
            df_pitch_spec_pitch = df_pitch_spec_name[df_pitch_spec_name[tagged].str.contains(pitch)]
            df_pitch_spec = df_pitch_spec_pitch[spin]
            df_pitch_spec = df_pitch_spec.reset_index()
            df_pitch_spec = df_pitch_spec.drop(['index'], axis=1)
            plt.plot(df_pitch_spec, label = pitch)
            plt.xlabel('# of pitches thrown')
            plt.ylabel('MPH')
            plt.title(name + '\nSpin Rate (in RPM)')
            plt.legend()
        plt.savefig(game_id + slash + 'Pitching' + slash + name + slash + name + pdf_endings_pitching[1], bbox_inches='tight')
        plt.show()

        
    # ---------- Tilt Histogram ---------
    for name in df_alabama_pitchers_index:
        df_tilt = df_alabama[df_alabama[pitcher].str.contains(name)]
        df_tilt = df_tilt[[tagged, tilt]]
        df_tilt = df_tilt.dropna()
        df_tilt = df_tilt.sort_values(tilt)

        fastball = df_tilt[(df_tilt[tagged] == '4-Fastball') | (df_tilt[tagged] == '2-Fastball')]
        changeup = df_tilt[df_tilt[tagged] == 'ChangeUp']
        slider = df_tilt[df_tilt[tagged] == 'Slider']
        curveball = df_tilt[df_tilt[tagged] == 'Curveball']

        fig, axs = plt.subplots(2, 2, figsize = (10,10))
        axs[0, 0].hist(fastball[tilt])
        axs[0, 0].set_title('Fastball')
        axs[0, 1].hist(changeup[tilt])
        axs[0, 1].set_title('Changeup')
        axs[1, 0].hist(slider[tilt])
        axs[1, 0].set_title('Slider')
        axs[1, 1].hist(curveball[tilt])
        axs[1, 1].set_title('Curveball')

        for ax in axs.flat:
            ax.set(xlabel='Tilt', ylabel='Count')
            plt.xticks(rotation=45)

        plt.savefig(game_id + slash + 'Pitching' + slash + name + slash + name + pdf_endings_pitching[2], bbox_inches='tight')
        plt.show()
        
    # ---------- Pitches by type by player Report ---------
    for name in df_alabama_pitchers_index:
        for pitch in df_pitches:
            df_pitch_spec_name = df[df[pitcher].str.contains(name)]
            frame = df_pitch_spec_name[[PlateLocHeight, PlateLocSide, tagged]]
            df_pitch_spec_name = frame[frame[tagged].str.contains(pitch)]    
            plt.scatter(df_pitch_spec_name[PlateLocSide], df_pitch_spec_name[PlateLocHeight], label=pitch)
            plt.axhline(y=1.5, linewidth=2, color='k')
            plt.axhline(y=3.6, linewidth=2, color='k')
            plt.axvline(x=(2/3), linewidth=2, color='k')
            plt.axvline(x=(-2/3), linewidth=2, color='k')
            plt.title(name + '\nPitching Map \n Over Plate')
            plt.xlabel('<L>     Side in Feet     <R>')
            plt.ylabel('Height in Feet')
            plt.legend()
            plt.xlim([-2.5, 2.5])
            plt.ylim([0, 5])      
        plt.savefig(game_id + slash + 'Pitching' + slash + name + slash + name + pdf_endings_pitching[3], bbox_inches='tight')
        plt.show() 

    # ---------- Release Point Report (XY) ---------
    for name in df_alabama_pitchers_index:
        for pitch in df_pitches:
            rel_name = df[df[pitcher].str.contains(name)]
            rel_pitch = rel_name[rel_name[tagged].str.contains(pitch)]
            rel_data = rel_pitch.reset_index()
            rel_data_spec = rel_data.drop(['index'], axis=1)
            plt.scatter(rel_data_spec[rel_side], rel_data_spec[rel_height], label=pitch)
            plt.xlabel('<L>     Side in Feet     <R>')
            plt.ylabel('Height in Feet')
            plt.title(name + '\nRelease Point (XY) \n From Mound')
            plt.legend()
        plt.savefig(game_id + slash + 'Pitching' + slash + name + slash + name + pdf_endings_pitching[4], bbox_inches='tight')
        plt.show()

    # ---------- Hard Hit Report ---------
    for name in df_alabama_pitchers_index:
        pitcher_df = df[df[pitcher].str.contains(name)]
        hard_hit_df = pitcher_df[pitcher_df[hardhit].str.contains('1')]
        not_hard_hit_df = pitcher_df[pitcher_df[hardhit].str.contains('0')]
        plt.scatter(pitcher_df[PlateLocSide], pitcher_df[PlateLocHeight], marker='.', label='Pitch', c='g')
        plt.scatter(not_hard_hit_df[PlateLocSide], not_hard_hit_df[PlateLocHeight], marker='.', label='Hit', c='r')
        plt.scatter(hard_hit_df[PlateLocSide], hard_hit_df[PlateLocHeight], marker='^', label='Hard Hit', c='r')
        plt.axhline(y=1.5, linewidth=2, color='k')
        plt.axhline(y=3.6, linewidth=2, color='k')
        plt.axvline(x=(2/3), linewidth=2, color='k')
        plt.axvline(x=(-2/3), linewidth=2, color='k')
        plt.title(name + '\nHard Hit Chart')
        plt.xlabel('<L>     Side in Feet     <R>')
        plt.ylabel('Height in Feet')
        plt.xlim([-2.5, 2.5])
        plt.ylim([0, 5])
        plt.legend()
        plt.savefig(game_id + slash + 'Pitching' + slash + name + slash + name + pdf_endings_pitching[5], bbox_inches='tight')
        plt.show()

    # ---------- Release Point Report (Z) ---------
    for name in df_alabama_pitchers_index:
        rel_name = df[df[pitcher].str.contains(name)]
        rel_pitch = rel_name[rel_name[tagged].str.contains('Fastball')]
        rel_data = rel_pitch.reset_index()
        rel_data = rel_data.reset_index()
        rel_data_spec = rel_data.drop(['index'], axis=1)
        plt.plot(rel_data_spec['Extension'], rel_data_spec['level_0'], label='Fastball')
        plt.title(name + '\nRelease Point (Etension Over Time)')
        plt.xlabel('<PM = 0>     Length in Feet     <HP>')
        plt.ylabel('Pitch Number')
        plt.legend()
        plt.gca().invert_yaxis()
        plt.savefig(game_id + slash + 'Pitching' + slash + name + slash + name + pdf_endings_pitching[6], bbox_inches='tight')
        plt.show()

    # ----------- Batting Pitch Type (XY) ------------   
    for name in df_alabama_hitter_index:
        for pitch in df_pitches:
            hitter_df = df[df[batter].str.contains(name)]
            frame = hitter_df[[PlateLocHeight, PlateLocSide, tagged]]
            df_pitch_spec_name = frame[frame[tagged].str.contains(pitch)]    
            plt.scatter(df_pitch_spec_name[PlateLocSide], df_pitch_spec_name[PlateLocHeight], label=pitch)
            plt.axhline(y=1.5, linewidth=2, color='k')
            plt.axhline(y=3.6, linewidth=2, color='k')
            plt.axvline(x=(2/3), linewidth=2, color='k')
            plt.axvline(x=(-2/3), linewidth=2, color='k')
            plt.title(name + '\nPitching Map \n Over Plate')
            plt.xlabel('<L>     Side in Feet     <R>')
            plt.ylabel('Height in Feet')
            plt.legend()
            plt.xlim([-2.5, 2.5])
            plt.ylim([0, 5])
        plt.savefig(game_id + slash + 'Hitting' + slash + name + slash + name + pdf_endings_hitting[0], bbox_inches='tight')
        plt.show() 

    # ---------- Strike Zone Report ---------
    for name in df_alabama_hitter_index:
        frame = df[df[batter].str.contains(name)]
        frame = frame[[PlateLocHeight, PlateLocSide, pitchcall]]
        df_ball = frame[frame[pitchcall].str.contains('BallCalled')]
        df_strike = frame[frame[pitchcall].str.contains('StrikeCalled')]
        plt.scatter(df_ball[PlateLocSide], df_ball[PlateLocHeight], label='Ball Called')
        plt.scatter(df_strike[PlateLocSide], df_strike[PlateLocHeight], label='Strike Called')
        plt.axhline(y=1.5, linewidth=2, color='k')
        plt.axhline(y=3.6, linewidth=2, color='k')
        plt.axvline(x=(2/3), linewidth=2, color='k')
        plt.axvline(x=(-2/3), linewidth=2, color='k')
        plt.legend()
        plt.title(name + '\nBall / Strike Zone Chart')
        plt.xlabel('<L>     Side in Feet     <R>')
        plt.ylabel('Height in Feet')
        plt.xlim([-2.5, 2.5])
        plt.ylim([0, 5]) 
        plt.savefig(game_id + slash + 'Hitting' + slash + name + slash + name + pdf_endings_hitting[1], bbox_inches='tight')
        plt.show()

    # ---------- Hard Hit Report ---------
    for name in df_alabama_hitter_index:
        batter_df = df[df[batter].str.contains(name)]
        hard_hit_df = batter_df[batter_df[hardhit].str.contains('1')]
        not_hard_hit_df = batter_df[batter_df[hardhit].str.contains('0')]
        plt.scatter(batter_df[PlateLocSide], batter_df[PlateLocHeight], marker='.', label='Pitch', c='g')
        plt.scatter(not_hard_hit_df[PlateLocSide], not_hard_hit_df[PlateLocHeight], marker='.', label='Hit', c='r')
        plt.scatter(hard_hit_df[PlateLocSide], hard_hit_df[PlateLocHeight], marker='^', label='Hard Hit', c='r')
        plt.axhline(y=1.5, linewidth=2, color='k')
        plt.axhline(y=3.6, linewidth=2, color='k')
        plt.axvline(x=(2/3), linewidth=2, color='k')
        plt.axvline(x=(-2/3), linewidth=2, color='k')
        plt.title(name + '\nHard Hit Chart')
        plt.xlabel('<L>     Side in Feet     <R>')
        plt.ylabel('Height in Feet')
        plt.xlim([-2.5, 2.5])
        plt.ylim([0, 5])
        plt.legend()
        plt.savefig(game_id + slash + 'Hitting' + slash + name + slash + name + pdf_endings_hitting[2], bbox_inches='tight')
        plt.show()    

    # ---------- Merge and Delete the Srike Zone PDF Files ------------- 
    merger = PdfFileMerger()
    for pdf_ending in pdf_endings_file:
        pdf_name = game_id + slash + 'Game' + slash + pdf_ending
        merger.append(pdf_name)
    merger.write(game_id + slash + 'Game' + slash + game_id + ' - Strike Zone Final.pdf')
    merger.close()

    for pdf_ending in pdf_endings_file:
        pdf_name = game_id + slash + 'Game' + slash + pdf_ending   
        os.remove(pdf_name)
            
    # ---------- Merge and Delete the Pitching PDF Files ------------- 
    for name in df_alabama_pitchers_index:
        merger = PdfFileMerger()
        for pdf_ending in pdf_endings_pitching:
            pdf_name = game_id + slash + 'Pitching' + slash + name + slash + name + pdf_ending
            merger.append(pdf_name)
        merger.write(game_id + slash + 'Pitching' + slash + name + slash + name + " result.pdf")
        merger.close()

    for name in df_alabama_pitchers_index:
        for pdf_ending in pdf_endings_pitching:
            pdf_name = game_id + slash + 'Pitching' + slash + name + slash + name + pdf_ending    
            os.remove(pdf_name)
            
    # ---------- Merge and Delete the Hitting PDF Files ------------- 
    for name in df_alabama_hitter_index:
        merger = PdfFileMerger()
        for pdf_ending in pdf_endings_hitting:
            pdf_name = game_id + slash + 'Hitting' + slash + name + slash + name + pdf_ending
            merger.append(pdf_name)
        merger.write(game_id + slash + 'Hitting' + slash + name + slash + name + " result.pdf")
        merger.close()

    for name in df_alabama_hitter_index:
        for pdf_ending in pdf_endings_hitting:
            pdf_name = game_id + slash + 'Hitting' + slash + name + slash + name + pdf_ending    
            os.remove(pdf_name)    
             
    # ---------- Save then Delete Raw CSV to records as game_id --------------
    df.set_index(PitchNo, inplace=True)
    df.to_csv(game_id + slash + 'Game' + slash + game_id + ' - Data.csv')
    df.to_csv('Archived Data' + slash + game_id + ' - Data.csv')

    # ---------- Delete Extra Files ---------2
    count = 0
    while count <= 30: 
        count += 1
        try: shutil.rmtree(game_id + slash + 'Pitching' + slash + 'pitcher_' + str(count))
        except: pass
        
    count = 0
    while count <= 30: 
        count += 1       
        try: shutil.rmtree(game_id + slash + 'Hitting' + slash + 'batter_' + str(count))
        except: pass

    # --------- Manage Files ---------
    shutil.make_archive(path + game_id, 'zip', game_id)
    shutil.rmtree(game_id)
    
    if remove_raw_data == True: os.remove(data)
    elif remove_raw_data == False: pass
    
postgame_report(remove_raw_data = False)